In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

from IPython.display import clear_output
from datetime import datetime

# = = = = = = = = = = = = = = =

import requests
import json

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy


data = {'getAPIKeyForUser': {'catalog': 'tecdocsw',
                             'username': '186299u15',
                             'password': 'APMTeDoW14'}}

b = 0
while True:
    b += 1

    try:
        resp = requests.post('https://webservice.tecalliance.services/webcat30/v1/services/WebCat30WS.jsonEndpoint',
                             data=json.dumps(data),
                             headers=get_header(),
                             #proxies=get_proxy(),
                             timeout=(10, 10))

        if resp.status_code == 200:
            json_ = resp.json()

            if json_['status'] == 200:
                key = json_['apiKey']
                break
    except KeyboardInterrupt:
        break
    except:
        continue


# = = = = = = = = = = = = = = =

import pandas as pd

# = = = = = = = = = = = = = = =

import math

# = = = = = = = = = = = = = = =

output_correct, output_error = pd.DataFrame(), pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel('./file/input.xlsx',
                       header=0,
                       dtype=str).fillna('')

input_['No'] = input_['No'].astype(int)

print(f'总数量：{len(input_)}')
print()

# = = = = = = = = = = = = = = =

work = Queue()
for a in range(len(input_)):
    work.put_nowait(a)
    
# = = = = = = = = = = = = = = =

def crawler():
    global output_correct, output_error, key

    while not work.empty():
        a = work.get_nowait()

        # = = = = = = = = = = = = = = =

        crawler_status = 'error'

        # = = = = = = = = = = = = = = =

        try:
            url = input_.loc[a, 'Url']
            
            dict_param = {}
            list_param = url.split('#@')[0].split('?')[1].split('&') if '?' in url else []
            for param in list_param:
                dict_param[param.split('=')[0]] = param.split('=')[1]

            data = {'getArticles': {'articleCountry': 'DE',
                                    'articleStatusIds': [],
                                    'criteriaFilters': [],
                                    'dataSupplierIds': [int(dict_param['brands'])],
                                    'filterQueries':  ['(dataSupplierId NOT IN (4978,4982))'],
                                    'genericArticleIds': [int(dict_param['groups'])],
                                    'includeAccessoryArticles': False,
                                    'includeAll': False,
                                    'includeArticleCriteria': True,
                                    'includeArticleLogisticsCriteria': True,
                                    'includeArticleStatusFacets': True,
                                    'includeArticleText': True,
                                    'includeComparableNumbers': True,
                                    'includeCriteriaFacets': True,
                                    'includeDataSupplierFacets': True,
                                    'includeGTINs': True,
                                    'includeGenericArticleFacets': True,
                                    'includePartsListArticles': False,
                                    'includeImages': True,
                                    'includeLinkages': True,
                                    'includeLinks': True,
                                    'includeMisc': True,
                                    'includeOEMNumbers': True,
                                    'includePDFs': True,
                                    'includePartsListArticles': False,
                                    'includePrices': False,
                                    'includeReplacedByArticles': True,
                                    'includeReplacesArticles': True,
                                    'includeTradeNumbers': True,
                                    'lang': 'en',
                                    'linkagesPerPage': 100,
                                    'page': 1,
                                    'perPage': 1_000,
                                    'provider': 40_000,
                                    'sort': [{'field': 'articleNumber', 'direction': 'asc'}]}}

            b = 0
            while True:
                b += 1
                
                try:
                    resp = requests.post('https://webservice.tecalliance.services/pegasus-3-0/services/TecdocToCatDLB.jsonEndpoint',
                                         data=json.dumps(data),
                                         headers=get_header(**{'x-api-key': key}),
                                         #proxies=get_proxy(),
                                         timeout=(10, 10))
            
                    if resp.status_code == 200:
                        json_ = resp.json()
            
                        if json_['status'] == 401 and json_['statusText'].strip() == 'Access not allowed':
                            while True:
                                try:
                                    data = {'getAPIKeyForUser': {'catalog': 'tecdocsw',
                                                                 'username': '186299u15',
                                                                 'password': 'APMTeDoW14'}}
                                    
                                    resp = requests.post('https://webservice.tecalliance.services/webcat30/v1/services/WebCat30WS.jsonEndpoint',
                                                         data=json.dumps(data),
                                                         headers=get_header(),
                                                         #proxies=get_proxy(),
                                                         timeout=(10, 10))
                            
                                    if resp.status_code == 200:
                                        json_ = resp.json()
                            
                                        if json_['status'] == 200:
                                            key = json_['apiKey']
                                            break
                                except KeyboardInterrupt:
                                    break
                                except:
                                    continue
                        elif json_['status'] == 200:
                            break
                except KeyboardInterrupt:
                    break
                except:
                    continue

            # = = = = = = = = = = = = = = =

            count = json_['totalMatchingArticles']

            # = = = = = = = = = = = = = = =

            if count <= 0:
                raise

            # = = = = = = = = = = = = = = =

            df_temp = pd.DataFrame([{'No': input_.loc[a, 'No'],
                                     'Url': f'''https://web.tecalliance.net/tecdocsw/en/parts/search?brands={int(dict_param['brands'])}&groups={int(dict_param['groups'])}''',
                                     'SKU Count': count,
                                     'Page': math.ceil(count / 1_000)}])
            
            output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')
            
            # = = = = = = = = = = = = = = =

            crawler_status = 'ok'
        except:
            output_error = pd.concat([output_error, pd.DataFrame([input_.iloc[a]]).reset_index(drop=True)], ignore_index=True).fillna('')

        # = = = = = = = = = = = = = = =

        if work.qsize() % 1_000 == 0 and work.qsize() != 0:
            clear_output()

        print(f'''[状态：{crawler_status}，尝试次数：{b}] - {input_.loc[a, 'No']}.{input_.loc[a, 'Url']}\n[剩余数量：{work.qsize()}] - [当前时间：{datetime.now().strftime('%H:%M:%S')}]\n''')

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(5):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
if not output_correct.empty:
    output_correct = output_correct.sort_values(by=['No'],
                                                ascending=[True],
                                                ignore_index=True)
    output_correct.to_excel(f'''./file/1.page-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)
if not output_error.empty:
    output_error = output_error.sort_values(by=['No'],
                                            ascending=[True],
                                            ignore_index=True)
    output_error.to_excel(f'''./file/page_error.xlsx''', index=False)
    print('爬虫存在error')
    print()
print('Done ~')

总数量：2

[状态：ok，尝试次数：1] - 2.https://web.tecalliance.net/tecdocsw/en/parts/search?brands=244&groups=1361#@brc/search:Search%2520by%2520brands%2520and%2520product%2520groups;brands:244;groups:1361
[剩余数量：0] - [当前时间：15:57:11]

[状态：ok，尝试次数：1] - 1.https://web.tecalliance.net/tecdocsw/en/parts/search?brands=101&groups=247#@brc/search:Search%2520by%2520brands%2520and%2520product%2520groups;brands:101;groups:247
[剩余数量：0] - [当前时间：15:57:15]

输出ing...

Done ~
